# Анализ показателей YouTube канала в соотвествии с методикой AIDA

## Цель исследования

Осуществить выгрузку метрик YouTube канала с помощью API и подготовить дашборд в Looker Studio в соответствии с методикой AIDA. На следующем этапе данная информация будет использоваться для расчета ROMI.

## Задачи исследования
- Импорт данных YouTube канала в разрезе видео роликов и дат в Google Sheets;
- Подготовить дашборд в Looker Studio;


### Подготовка

In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta


import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

In [2]:
# ключ API
api_key = '---'

# айди канала для анализа
channel_id = '---'

youtube = build('youtube', 'v3', developerKey = api_key)

### Выгрузка данных в разбивке по видео

Выгрузим отчет с разбивкой по видео за период в 180 дней с помощью YouTube Analytics API

In [ ]:
# -*- coding: utf-8 -*-

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'client_secret.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
      **kwargs
  ).execute()

  print(response)
  return response

if __name__ == '__main__':
  # Disable OAuthlib's HTTPs verification when running locally.
  # *DO NOT* leave this option enabled when running in production.
  os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

  youtubeAnalytics = get_service()
  stat = execute_api_request(
      youtubeAnalytics.reports().query,
      ids='channel==---',
      dimensions='video',
      startDate=(datetime.today() - timedelta(days=180)).strftime('%Y-%m-%d'),
      endDate=datetime.today().strftime('%Y-%m-%d'),
      maxResults='200',
      sort='-views',
      metrics='views,estimatedMinutesWatched,averageViewDuration,likes,dislikes,comments,shares,subscribersGained,cardImpressions,cardClicks,cardTeaserImpressions,cardTeaserClicks'
  )

Подготовим датафрейм с данными. Также выгрузим информацию о названии роликов и дате их загрузки.

In [4]:
headers = ['id','views','estimatedMinutesWatched','averageViewDuration','likes','dislikes','comments','shares','subscribersGained','cardImpressions','cardClicks','cardTeaserImpressions','cardTeaserClicks']
by_video = pd.DataFrame(data=stat['rows'],columns=headers)

In [5]:
def get_video_details(youtube, video_ids):
    all_video_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
                    part='snippet,statistics',
                    id=','.join(video_ids[i:i+50]))
        response = request.execute()
        
        for video in response['items']:
            video_stats = dict(id=video['id'], title = video['snippet']['title'],
                               published_date = video['snippet']['publishedAt'])
            all_video_stats.append(video_stats)
    
    return all_video_stats

In [6]:
video_titles = pd.DataFrame(get_video_details(youtube, by_video['id']))
video_titles.head()

,id,title,published_date
0,CMfLxO5Zrgg,Терминал B — Шереметьево Паркинг | Моушн-дизай...,2019-04-15T11:02:54Z
1,Nnh4VkN9uc8,Служебное огнестрельное оружие в работе охранн...,2018-10-02T21:27:14Z
2,KIcqcGLl4E4,Служебное огнестрельное оружие в работе охранн...,2018-12-01T21:35:39Z
3,9rn6HHnzCn4,Правила противопожарной безопасности в ТЦ | Ду...,2018-09-15T07:39:16Z
4,xo4DKaouLto,Шокер для охранников. Как использовать спецсре...,2018-08-25T21:21:40Z


In [7]:
by_video = by_video.merge(video_titles, on='id')

In [8]:
by_video['published_date'] = pd.to_datetime(by_video['published_date'])
by_video['published_date'] = by_video['published_date'].dt.strftime('%Y-%m-%d')

In [9]:
by_video.head()

,id,views,estimatedMinutesWatched,averageViewDuration,likes,dislikes,comments,shares,subscribersGained,cardImpressions,cardClicks,cardTeaserImpressions,cardTeaserClicks,title,published_date
0,CMfLxO5Zrgg,1632,2010,73,6,0,0,18,3,0,0,0,0,Терминал B — Шереметьево Паркинг | Моушн-дизай...,2019-04-15
1,Nnh4VkN9uc8,1354,2234,99,32,2,0,16,8,0,0,0,0,Служебное огнестрельное оружие в работе охранн...,2018-10-02
2,KIcqcGLl4E4,805,1670,124,21,0,1,9,8,0,0,0,0,Служебное огнестрельное оружие в работе охранн...,2018-12-01
3,9rn6HHnzCn4,292,426,87,8,0,0,4,2,0,0,39,0,Правила противопожарной безопасности в ТЦ | Ду...,2018-09-15
4,xo4DKaouLto,199,338,102,5,0,1,4,2,0,0,0,0,Шокер для охранников. Как использовать спецсре...,2018-08-25


### Выгрузка данных в разбивке по датам

Аналогичным образом выгрузим отчет в разрезе дней за период в предшествующие 180 дней.

In [ ]:
if __name__ == '__main__':
  # Disable OAuthlib's HTTPs verification when running locally.
  # *DO NOT* leave this option enabled when running in production.
  os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

  youtubeAnalytics = get_service()
  stat = execute_api_request(
      youtubeAnalytics.reports().query,
      ids='channel==---',
      dimensions='day',
      startDate=(datetime.today() - timedelta(days=180)).strftime('%Y-%m-%d'),
      endDate=datetime.today().strftime('%Y-%m-%d'),
      metrics='views,estimatedMinutesWatched,averageViewDuration,likes,dislikes,comments,shares,subscribersGained,cardImpressions,cardClicks,cardTeaserImpressions,cardTeaserClicks'
  )

In [11]:
headers = ['date','views','estimatedMinutesWatched','averageViewDuration','likes','dislikes','comments','shares','subscribersGained','cardImpressions','cardClicks','cardTeaserImpressions','cardTeaserClicks']
by_date = pd.DataFrame(data=stat['rows'],columns=headers)

In [12]:
by_date.head()

,date,views,estimatedMinutesWatched,averageViewDuration,likes,dislikes,comments,shares,subscribersGained,cardImpressions,cardClicks,cardTeaserImpressions,cardTeaserClicks
0,2022-05-26,26,36,84,1,0,0,1,1,0,0,3,0
1,2022-05-27,49,56,69,0,0,0,3,0,0,0,4,0
2,2022-05-28,35,55,94,1,0,0,2,1,0,0,1,0
3,2022-05-29,33,47,87,0,0,0,0,0,0,0,2,0
4,2022-05-30,40,51,77,0,0,0,0,1,0,0,0,0


### Экспорт таблиц в Google Sheets

In [13]:
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('client_key.json', scope)
client = gspread.authorize(credentials)


gc = gspread.authorize(credentials)

In [14]:
sh = gc.open('yt_dashboard')

In [15]:
video_worksheet = sh.add_worksheet(title="video_data", rows=100, cols=20)
date_worksheet = sh.add_worksheet(title="date_data", rows=100, cols=20)

In [16]:
video_worksheet.update([by_video.columns.values.tolist()] + by_video.values.tolist())
date_worksheet.update([by_date.columns.values.tolist()] + by_date.values.tolist())

{'spreadsheetId': '1luzzPeoj3LzkU6ApoZwiCWj9wqvOmF1sJw-1bxdfUTU',
 'updatedRange': 'date_data!A1:M179',
 'updatedRows': 179,
 'updatedColumns': 13,
 'updatedCells': 2327}

## Результаты

В ходе работы:
- получен доступ к информации о роликах канала с помощью YouTube Analytics API;
- выгружены метрики в разрезе видео и дней за период в 180 дней и переданы в Looker Studio;
- подготовлен [дашборд](https://datastudio.google.com/reporting/d8c8a0af-4a13-4a02-9141-f3007a7d13b2) в соответствии с методикой AIDA.